<a href="https://colab.research.google.com/github/gitHubAndyLee2020/GitHub_Copilot_Clone/blob/main/github_copilot_clone_project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import numpy as np
import os
import string
import random

In [ ]:
from google.colab import files

# This will prompt you to select a file from your local machine
uploaded = files.upload()

# You can then access the uploaded files like this:
for filename in uploaded.keys():
  print(f"User uploaded file {filename} with length {len(uploaded[filename])} bytes")

Saving leetcode_solutions.csv to leetcode_solutions.csv
User uploaded file leetcode_solutions.csv with length 1024681 bytes


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
leetcode_solutions = pd.read_csv('leetcode_solutions.csv')
leetcode_solutions = leetcode_solutions['Answer'].to_numpy()

TERMINATING_TOKEN = "endseq"

# Append the terminating token to each solution
leetcode_solutions = [f"{solution} {TERMINATING_TOKEN}" for solution in leetcode_solutions]

# Convert back to NumPy array
leetcode_solutions = np.array(leetcode_solutions)

# Show the DataFrame
print(f"{len(leetcode_solutions)} leetcode solutions in C++")
print(leetcode_solutions[0])

1567 leetcode solutions in C++
class Solution {
 public:
  vector<int> twoSum(vector<int>& nums, int target) {
    unordered_map<int, int> numToIndex;

    for (int i = 0; i < nums.size(); ++i) {
      if (const auto it = numToIndex.find(target - nums[i]);
          it != cend(numToIndex))
        return {it->second, i};
      numToIndex[nums[i]] = i;
    }

    throw;
  }
}; endseq


In [ ]:
import tensorflow as tf
import re

class CustomCodeVectorization(tf.keras.layers.Layer):
    def __init__(self, max_tokens=5000, output_sequence_length=None):
        super(CustomCodeVectorization, self).__init__()
        self.max_tokens = max_tokens
        self.output_sequence_length = output_sequence_length
        self.tokenizer = re.compile(r"([a-zA-Z_]\w+|\d+|==|<=|>=|&&|\|\||[^a-zA-Z\d\s])")
        self.vocab = {"": 0, "[UNK]": 1}  # Reserve 0 for padding, 1 for unknown tokens
        self.vocab_size = 2

    def adapt(self, texts):
        all_tokens = []
        for text in texts:
            tokens = self.tokenizer.findall(text)
            all_tokens.extend(tokens)

        unique_tokens = list(set(all_tokens))
        if len(unique_tokens) > self.max_tokens - self.vocab_size:
            unique_tokens = sorted(unique_tokens, key=lambda t: all_tokens.count(t), reverse=True)[:self.max_tokens-self.vocab_size]

        for i, token in enumerate(unique_tokens):
            self.vocab[token] = i + self.vocab_size  # Start from 2 because 0 and 1 are reserved

        self.vocab_size = len(self.vocab)
        self.token_to_id = self.vocab
        self.id_to_token = {v: k for k, v in self.vocab.items()}

    def call(self, inputs):
        tokens = self.tokenizer.findall(inputs)
        token_ids = [self.token_to_id.get(token, 1) for token in tokens]  # 1 is the ID for "[UNK]"
        if self.output_sequence_length:
            token_ids = token_ids[:self.output_sequence_length]
            token_ids += [0] * (self.output_sequence_length - len(token_ids))  # Pad with zeros
        return tf.convert_to_tensor(token_ids)

    # Add get_vocabulary method
    def get_vocabulary(self):
        return [self.id_to_token[i] for i in range(self.vocab_size)]

In [ ]:
# Initialize CustomCodeVectorization layer
custom_vectorize_layer = CustomCodeVectorization(
    max_tokens=1000,
    output_sequence_length=None  # Dynamic sequence length
)

# Adapt the layer to the C++ code snippets
custom_vectorize_layer.adapt(leetcode_solutions)

# Vectorize the code snippets
vectorized_snippets = [custom_vectorize_layer(snippet) for snippet in leetcode_solutions]

# Display tokenized and vectorized versions
for i, snippet in enumerate(leetcode_solutions[:5]):
    print(f"Original Code Snippet: {snippet}")

    # Tokenized version
    tokenized_version = " ".join([custom_vectorize_layer.id_to_token[token] for token in vectorized_snippets[i].numpy() if token != 0])
    print(f"Tokenized Version: {tokenized_version}")

    # Vectorized version
    print(f"Vectorized Version: {vectorized_snippets[i].numpy()}")
    print("---")

Original Code Snippet: class Solution {
 public:
  vector<int> twoSum(vector<int>& nums, int target) {
    unordered_map<int, int> numToIndex;

    for (int i = 0; i < nums.size(); ++i) {
      if (const auto it = numToIndex.find(target - nums[i]);
          it != cend(numToIndex))
        return {it->second, i};
      numToIndex[nums[i]] = i;
    }

    throw;
  }
}; endseq
Tokenized Version: class Solution { public : vector < int > [UNK] ( vector < int > & nums , int target ) { unordered_map < int , int > numToIndex ; for ( int = 0 ; < nums . size ( ) ; + + ) { if ( const auto it = numToIndex . find ( target - nums [ ] ) ; it ! = cend ( numToIndex ) ) return { it - > second , } ; numToIndex [ nums [ ] ] = ; } throw ; } } ; endseq
Vectorized Version: [ 32  36  13  34  21  20  12   5  11   1   4  20  12   5  11  26  35   7
   5  68   3  13  99  12   5   7   5  11 412   2  27   4   5   6  17   2
  12  35  15  37   4   3   2  10  10   3  13  25   4  23  62  73   6 412
  15  90   4  68  1

In [ ]:
vocab = custom_vectorize_layer.get_vocabulary()

print(f"length of vocab: {len(vocab)}")

length of vocab: 1000


In [ ]:
batch_size = 128

# First, vectorize the leetcode_solutions numpy array using your custom_vectorize_layer
vectorized_solutions = [custom_vectorize_layer(snippet).numpy() for snippet in leetcode_solutions]

# Convert list of numpy arrays to a single numpy array
vectorized_solutions = np.array(vectorized_solutions)

# Step 1: Determine the maximum length among all sequences
max_length = max([len(seq) for seq in vectorized_solutions])

# Step 2: Pad each sequence to have this maximum length
vectorized_solutions_padded = [np.pad(seq, (0, max_length - len(seq)), 'constant') for seq in vectorized_solutions]

# Step 3: Convert the padded list to a NumPy array
vectorized_solutions_padded = np.array(vectorized_solutions_padded)

# Step 4: Create the TensorFlow dataset
text_ds = tf.data.Dataset.from_tensor_slices(vectorized_solutions_padded)

# Shuffle, batch, and prefetch
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)

<ipython-input-80-c1faee73ad2d>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectorized_solutions = np.array(vectorized_solutions)


In [ ]:
print("max sequence length:", max_length)

max sequence length: 1546


In [ ]:
# Sample a batch and print out the content
for batch in text_ds.take(1):
    for i, sample in enumerate(batch):
        if i >= 10:  # limit the number of samples printed to 10
            break
        token_list = [custom_vectorize_layer.id_to_token[token] for token in sample.numpy() if token != 0]
        code_str = " ".join(token_list)
        print(f"Sample {i+1}: {code_str}")

Sample 1: class Solution { public : bool [UNK] ( vector < vector < int > > & matrix , int target ) { if ( matrix . empty ( ) ) return false ; const int = matrix . size ( ) ; const int = matrix [ 0 ] . size ( ) ; int = 0 ; int = * ; while ( < ) { const int mid = ( + ) / 2 ; const int = mid / ; const int = mid % ; if ( matrix [ ] [ ] == target ) return true ; if ( matrix [ ] [ ] < target ) = mid + 1 ; else = mid ; } return false ; } } ; endseq
Sample 2: class Solution { public : [UNK] [UNK] ( [UNK] ) { [UNK] ans = 0 ; for ( int = 0 ; < 32 ; + + ) if ( > > & 1 ) ans | = 1 < < 31 - ; return ans ; } } ; endseq
Sample 3: [UNK] ( word1 : str , word2 : str ) endseq
Sample 4: class Solution { public : bool [UNK] ( string , string ) { vector < int > [UNK] ( 128 ) ; vector < int > [UNK] ( 128 ) ; for ( int = 0 ; < . length ( ) ; + + ) { if ( [UNK] [ [ ] ] ! = [UNK] [ [ ] ] ) return false ; [UNK] [ [ ] ] = + 1 ; [UNK] [ [ ] ] = + 1 ; } return true ; } } ; endseq
Sample 5: class Solution { public :

In [ ]:
def prepare_lm_inputs_labels(vectorized_text):
    x = vectorized_text[:, :-1]  # omit last token
    y = vectorized_text[:, 1:]  # omit first token (shift by 1)
    return x, y

# Assuming text_ds is your batched Dataset of vectorized sequences
text_ds = text_ds.map(prepare_lm_inputs_labels)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# To print samples from the batched text_ds
for batch_x, batch_y in text_ds.take(1):
    for i, (sample_x, sample_y) in enumerate(zip(batch_x, batch_y)):
        if i >= 10:  # limit the number of samples printed to 10
            break
        token_list_x = [custom_vectorize_layer.id_to_token[token] for token in sample_x.numpy() if token != 0]
        token_list_y = [custom_vectorize_layer.id_to_token[token] for token in sample_y.numpy() if token != 0]

        code_str_x = " ".join(token_list_x)
        code_str_y = " ".join(token_list_y)

        print(f"Sample {i+1} X: {code_str_x}")
        print(f"Sample {i+1} Y: {code_str_y}")
        print("---")

Sample 1 X: class Solution { public : vector < vector < int > > [UNK] ( int , int ) { vector < vector < int > > ans ; dfs ( , , 1 , { } , ans ) ; return ans ; } private : void dfs ( int , int , int , vector < int > && path , vector < vector < int > > & ans ) { if ( == 0 && == 0 ) { ans . push_back ( path ) ; return ; } if ( == 0 || <= 0 ) return ; for ( int = ; <= 9 ; + + ) { path . push_back ( ) ; dfs ( - 1 , - , + 1 , move ( path ) , ans ) ; path . pop_back ( ) ; } } } ; endseq
Sample 1 Y: Solution { public : vector < vector < int > > [UNK] ( int , int ) { vector < vector < int > > ans ; dfs ( , , 1 , { } , ans ) ; return ans ; } private : void dfs ( int , int , int , vector < int > && path , vector < vector < int > > & ans ) { if ( == 0 && == 0 ) { ans . push_back ( path ) ; return ; } if ( == 0 || <= 0 ) return ; for ( int = ; <= 9 ; + + ) { path . push_back ( ) ; dfs ( - 1 , - , + 1 , move ( path ) , ans ) ; path . pop_back ( ) ; } } } ; endseq
---
Sample 2 X: class Solution { pub

In [ ]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
  i = tf.range(n_dest)[:, None]
  j = tf.range(n_src)
  m = i >= j - n_src + n_dest
  mask = tf.cast(m, dtype)
  mask = tf.reshape(mask, [1, n_dest, n_src])
  mult = tf.concat(
      [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
  )
  return tf.tile(mask, mult)

class TransformerBlock(layers.Layer):
  def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
    super().__init__()
    self.att = layers.MultiHeadAttention(num_heads, embed_dim)
    self.ffn = keras.Sequential(
        [layers.Dense(ff_dim, activation='relu'), layers.Dense(embed_dim),]
    )
    self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
    self.dropout1 = layers.Dropout(rate)
    self.dropout2 = layers.Dropout(rate)

  def call(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size = input_shape[0]
    seq_len = input_shape[1]
    causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
    attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
    attention_output = self.dropout1(attention_output)
    out1 = self.layernorm1(inputs + attention_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
  def __init__(self, maxlen, vocab_size, embed_dim):
    super().__init__()
    self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
    self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

  def call(self, x):
    maxlen = tf.shape(x)[-1]
    positions = tf.range(start=0, limit=maxlen, delta=1)
    positions = self.pos_emb(positions)
    x = self.token_emb(x)
    return x + positions

In [ ]:
vocab_size = 1000
maxlen = 1545
embed_dim = 256
num_heads = 2
feed_forward_dim = 256

def create_model():
  inputs = layers.Input(shape=(maxlen,), dtype=tf.int32)
  embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
  x = embedding_layer(inputs)
  transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
  x = transformer_block(x)
  outputs = layers.Dense(vocab_size)(x)
  model = keras.Model(inputs=inputs, outputs=[outputs, x])
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(
      "adam", loss=[loss_fn, None],
  )
  return model

In [ ]:
class TextGenerator(keras.callbacks.Callback):
  def __init__(
      self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
  ):
    self.max_tokens = max_tokens
    self.start_tokens = start_tokens
    self.index_to_word = index_to_word
    self.print_every = print_every
    self.k = top_k

  def sample_from(self, logits):
    logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
    indices = np.asarray(indices).astype("int32")
    preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
    preds = np.asarray(preds).astype("float32")
    return np.random.choice(indices, p=preds)

  def detokenize(self, number):
    return self.index_to_word[number]

  def on_epoch_end(self, epoch, logs=None):
    start_tokens = [_ for _ in self.start_tokens]
    if (epoch + 1) % self.print_every != 0:
      return
    num_tokens_generated = 0
    tokens_generated = []
    while num_tokens_generated <= self.max_tokens:
      pad_len = maxlen - len(start_tokens)
      sample_index = len(start_tokens) - 1
      if pad_len < 0:
        x = start_tokens[:maxlen]
        sample_index = maxlen - 1
      elif pad_len > 0:
        x = start_tokens + [0] * pad_len
      else:
        x = start_tokens
      x = np.array([x])
      y, _ = self.model.predict(x)
      sample_token = self.sample_from(y[0][sample_index])
      tokens_generated.append(sample_token)
      start_tokens.append(sample_token)
      num_tokens_generated = len(tokens_generated)
    txt = " ".join(
        [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
    )
    print(f"generated text:\n{txt}\n")

word_to_index = {}
for index, word in enumerate(vocab):
  word_to_index[word] = index

start_prompt = """
class Solution {
  public:
    vector<int> twoSum(vector<int>& nums, int target) {
"""
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 40
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)

In [ ]:
model = create_model()

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1545)]            0         
                                                                 
 token_and_position_embeddin  (None, 1545, 256)        651520    
 g_2 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_2 (Transf  (None, 1545, 256)        658688    
 ormerBlock)                                                     
                                                                 
 dense_7 (Dense)             (None, 1545, 1000)        257000    
                                                                 
Total params: 1,567,208
Trainable params: 1,567,208
Non-trainable params: 0
_________________________________________________

In [ ]:
model.fit(text_ds, verbose=2, epochs=1, callbacks=[text_gen_callback])

1/1 [==============================] - 0s 53ms/step
generated text:
class Solution { [UNK] [UNK] [UNK] [UNK] int [UNK] { [UNK] ; [UNK] ( ( { int if 2 [UNK] = = + ) ; ( ( [UNK] (                      

13/13 - 116s - loss: 1.6478 - dense_7_loss: 1.6478 - 116s/epoch - 9s/step


In [ ]:
CURRENT_MODEL_PATH = "github_copilot_clone_v2_epochs_25.h5"
NEXT_MODEL_PATH = "github_copilot_clone_v2_epochs_x.h5"

In [ ]:
model.save(CURRENT_MODEL_PATH)

In [ ]:
from google.colab import files

def download_file(file_name):
    # Download the file to the local computer
    files.download(file_name)

# Download the file
download_file(CURRENT_MODEL_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.models import load_model

custom_objects = {
    "TransformerBlock": TransformerBlock,
    "TokenAndPositionEmbedding": TokenAndPositionEmbedding
}

loaded_model = load_model(CURRENT_MODEL_PATH, custom_objects=custom_objects)

In [ ]:
loaded_model.fit(text_ds, verbose=2, epochs=5, callbacks=[text_gen_callback])
loaded_model.save(NEXT_MODEL_PATH)

Epoch 1/5
1/1 [==============================] - 0s 52ms/step
generated text:
class Solution { [UNK] [UNK] [UNK] [UNK] int [UNK] { int [UNK] [UNK] ( int [UNK] ) { const int [UNK] = 0 ; int [UNK] ( begin ( [UNK] ) ; / / / ' ' ' ' 000 ' , - - - 1 ] = [UNK] [UNK] -

13/13 - 116s - loss: 0.2779 - dense_7_loss: 0.2779 - 116s/epoch - 9s/step
Epoch 2/5
1/1 [==============================] - 0s 51ms/step
generated text:
class Solution { [UNK] [UNK] [UNK] [UNK] int [UNK] { int > & [UNK] , int = [UNK] ( ) ; int ans . size ( ) ) ; / / [UNK] = [UNK] ) ; for ( int ans = [UNK] + = 1 ' ) if ( ) +

13/13 - 114s - loss: 0.2754 - dense_7_loss: 0.2754 - 114s/epoch - 9s/step
Epoch 3/5
1/1 [==============================] - 0s 51ms/step
generated text:
class Solution { [UNK] [UNK] [UNK] [UNK] int [UNK] { string > [UNK] ( ) { return 0 ' 000 ' ' ' 000 ' ' ' ' ' ' 007 ; < string > ' ' ) ; if ( num ) { [UNK] ( ) ; } return true

13/13 - 114s - loss: 0.2726 - dense_7_loss: 0.2726 - 114s/epoch - 9s/step
Epoch 4/

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import re

class CodeGenerationModule(tf.Module):
    def __init__(self, model, vocab, word_to_index, maxlen=1545, top_k=10):
        self.model = model
        self.vocab = vocab
        self.word_to_index = word_to_index
        self.maxlen = maxlen
        self.top_k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.top_k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def format_cpp_code(self, token_list):
        formatted_code = []
        current_line = ''
        for token in token_list:
            current_line += token + ' '
            if token in ['{', '}', ';']:
                formatted_code.append(current_line.strip())
                current_line = ''
        return '\n'.join(formatted_code)

    def __call__(self, start_prompt, num_tokens_to_generate=40):
        start_tokens = [self.word_to_index.get(word, 1) for word in start_prompt.split()]
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= num_tokens_to_generate:
            pad_len = self.maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:self.maxlen]
                sample_index = self.maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x)
            logits = y[0][sample_index]
            sample_token = self.sample_from(logits)

            # Check for TERMINATING_TOKEN
            if self.vocab[sample_token] == "endseq":
                break

            tokens_generated.append(self.vocab[sample_token])
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)

        formatted_code = self.format_cpp_code(tokens_generated)
        return formatted_code

In [ ]:
# Create an instance of the TextGeneratorWrapper class
code_gen_module = CodeGenerationModule(loaded_model, vocab, word_to_index)

# Test the instance with a start prompt
start_prompt = """
class Solution {
  public:
    vector<int> twoSum(vector<int>& nums, int target) {
"""
generated_code = code_gen_module(start_prompt)
print(f"Generated code: {generated_code}")

1/1 [==============================] - 0s 53ms/step
Generated code: return [UNK] ) {
ListNode * [UNK] : [UNK] , [UNK] ( head ) {
const int ans = 0 ;
for ( int [UNK] ;
}
return [UNK] ( [UNK] ( int = 0 ) ;
return false ;


In [ ]:
code_samples = ""
for i in range(20):
  code_samples += (leetcode_solutions[i] + "\n")

In [ ]:
# Tokenize the code by splitting at whitespaces and some special characters
tokens = []
current_token = ''
for c in code_samples:
    if c.isspace() or c in '{}();':
        if current_token:
            tokens.append(current_token)
            current_token = ''
        if c in '{}();':
            tokens.append(c)
    else:
        current_token += c
if current_token:
    tokens.append(current_token)

# Initialize lists to store code snippets and their true predictions
snippets = []
true_predictions = []

i = 0  # Initialize index to zero
while len(snippets) < 10:  # Loop until you get 5 valid snippets and predictions
    if i + 80 > len(tokens):  # Exit loop if not enough tokens are left for a snippet and prediction
        break

    snippet_tokens = tokens[i:i + 40]
    prediction_tokens = tokens[i + 40:i + 80]

    # Check if the snippet contains 'endseq'
    if "endseq" not in snippet_tokens:

        # Check if the prediction contains 'endseq' and truncate if it does
        if "endseq" in prediction_tokens:
            prediction_tokens = prediction_tokens[:prediction_tokens.index("endseq")]

        # Append to the lists
        snippets.append(' '.join(snippet_tokens))
        true_predictions.append(' '.join(prediction_tokens))

        # Increment i for the next snippet and prediction
        i += 80
    else:
        # Skip 40 tokens and try again
        i += 40

# Now, 'snippets' contains the code snippets and 'true_predictions' contains their corresponding next tokens

In [ ]:
model_paths = [
    'github_copilot_clone_v2_epochs_5.h5',
    'github_copilot_clone_v2_epochs_10.h5',
    'github_copilot_clone_v2_epochs_15.h5',
    'github_copilot_clone_v2_epochs_20.h5',
    'github_copilot_clone_v2_epochs_25.h5',
]

models_generated_texts = {}

for model_path in model_paths:
  loaded_model = load_model(model_path, custom_objects=custom_objects)
  text_gen_module = CodeGenerationModule(loaded_model, vocab, word_to_index)
  models_generated_texts[model_path] = []
  for start_prompt in snippets:
    generated_text = text_gen_module(start_prompt)
    models_generated_texts[model_path].append(generated_text)

models_generated_texts = pd.DataFrame(models_generated_texts)
models_generated_texts['true_predictions'] = true_predictions
models_generated_texts.index = snippets

1/1 [==============================] - 0s 63ms/step


In [ ]:
import pandas as pd
from google.colab import files

# Create a DataFrame, assuming 'models_generated_texts' is your DataFrame
# models_generated_texts = pd.DataFrame({
#     'Column1': [1, 2, 3],
#     'Column2': ['a', 'b', 'c']
# })

# Save DataFrame to CSV file
models_generated_texts.to_csv('models_generated_texts.csv', index=False)

# Download the file to your local machine
files.download('models_generated_texts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
models_generated_texts.head()

,github_copilot_clone_v2_epochs_5.h5,github_copilot_clone_v2_epochs_10.h5,github_copilot_clone_v2_epochs_15.h5,github_copilot_clone_v2_epochs_20.h5,github_copilot_clone_v2_epochs_25.h5,true_predictions
"class Solution { public: vector<int> twoSum ( vector<int>& nums, int target ) { unordered_map<int, int> numToIndex ; for ( int i = 0 ; i < nums.size ( ) ; ++i ) { if ( const auto it = numToIndex.find","= 1 ] ;\n/ / / / / / / / / / dp [ + , - 1 ) ;",": void dfs ( const int , ans ;\nfor ( const in...",= - 1 ) ;\n/ / / 2 ;\n/ / / [UNK] = 1 ) ;\n}\n...,""" ;\n}\nreturn ans ;\n[UNK] = [UNK] ;\nwhile (...",[UNK] . push_back ( int [UNK] [UNK] ) ;\n[UNK]...,( target - nums[i] ) ; it != cend ( numToIndex...
l2 ) { ListNode dummy ( 0 ) ; ListNode* curr = &dummy ; int carry = 0 ; while ( l1 || l2 || carry ) { if ( l1 != nullptr ) { carry += l1->val ; l1,", ) = [UNK] ) ;",", int ans ) if ( ans = ;\n}\n}\n}\nreturn fals...","= ( ) return [UNK] ( ) + [UNK] , 0 ] [ ] [ 0 |...","- [UNK] ( begin ( ) , [UNK] [UNK] ( [UNK] ;\n+...",- > val ;\nelse if ( head - > next ) ;\n}\nret...,= l1->next ; } if ( l2 != nullptr ) { carry +=...
"public: double findMedianSortedArrays ( vector<int>& nums1, vector<int>& nums2 ) { const int n1 = nums1.size ( ) ; const int n2 = nums2.size ( ) ; if ( n1 > n2 ) return findMedianSortedArrays ( nums2, nums1 ) ; int","= ) {\n;\nreturn [UNK] = 0 ] - > [UNK] , ) ans...",> [UNK] . size ( < ;\nreturn false ;\nconst in...,= min ( ) ;\nreturn - 2 ] [ ] = 1 : = 0 ;\n+ [...,"[UNK] ) ;\nfor ( ) ;\n/ / / / / / "" , [UNK] . ...","[UNK] ( [UNK] , end ( [UNK] ) + 1 : [UNK] ) ) ...",l = 0 ; int r = n1 ; while ( l <= r ) { const ...
2 - partition1 ; const int maxLeft1 = partition1 == 0 ? INT_MIN : nums1[partition1 - 1] ; const int maxLeft2 = partition2 == 0 ? INT_MIN : nums2[partition2 - 1] ; const int minRight1 = partition1 == n1 ?,"- 1 ) {\nif ( [UNK] , , ) ) ;","1 , ) {\nconst string & [UNK] , [UNK] ( ) ) ;\...","[UNK] , 0 ;\n< 0 ;\nfor ( begin ( ) {",: [UNK] . resize ( int [UNK] ) return [UNK] ( ...,[UNK] = max ( string & ) if ( [UNK] ;\nfor ( [...,INT_MAX : nums1[partition1] ; const int minRig...
"maxLeft1, maxLeft2 ) + min ( minRight1, minRight2 ) ) * 0.5 : max ( maxLeft1, maxLeft2 ) ; else if ( maxLeft1 > minRight2 ) r = partition1 - 1 ; else l = partition1 + 1 ; }",if ( const int = [UNK] ( ) ;\n+ = 0 ;\n}\n;\n}...,;\nint = 0 ;\n}\n;\n+ ) ) ;\n+ 1 ) {\nif ( [UN...,else {\nfor ( int target < int [UNK] ;\n}\n}\n...,return true ;\nif ( num ;\nif ( root - [UNK] [...,return [UNK] - > & ans ;\n}\nreturn root - > l...,throw ; } } ;


- From examining the outputs of each model, it appears github_copilot_clone_v2_epochs_20.h5	have the most plausible output.
- github_copilot_clone_v2_epochs_20.h5 will be selected as the model for building the GitHub Copilor Clone Project demo.